# A3C for Kung Fu

## Part 0 - Instalar paquetes e importar librerias

### Instalar Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.4 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=f8ee303f1cae7d4aebc9028a468a65e00535b5cff75bcc949d72d4af96fce6f6
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:


### Importar librerias

In [2]:
import cv2
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.distributions as distributions
from torch.distributions import Categorical
import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.spaces import Box

## Part 1 - Construcción

### Crear la arquitectura de la red neuronal

In [3]:
class Network(nn.Module):

  def __init__(self, action_size):
    super(Network, self).__init__()

    # Definir la primera capa convolucional con 4 canales de entrada y 32 canales de salida,
    # un tamaño de kernel de 3x3, y un stride de 2.
    self.conv1 = torch.nn.Conv2d(in_channels=4, out_channels=32, kernel_size=(3, 3), stride=2)

    # Definir la segunda capa convolucional con 32 canales de entrada y 32 canales de salida,
    # un tamaño de kernel de 3x3, y un stride de 2.
    self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), stride=2)

    # Definir la tercera capa convolucional con 32 canales de entrada y 32 canales de salida,
    # un tamaño de kernel de 3x3, y un stride de 2.
    self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), stride=2)

    # Definir una capa de aplanado para convertir la salida de las capas convolucionales en un vector plano.
    self.flatten = torch.nn.Flatten()

    # Definir una capa totalmente conectada (fully connected) con 512 neuronas de entrada y 128 de salida.
    self.fc1 = torch.nn.Linear(512, 128)

    # Definir la capa de salida para los valores de acción con 128 neuronas de entrada y `action_size` de salida.
    self.fc2a = torch.nn.Linear(128, action_size)

    # Definir la capa de salida para el valor del estado con 128 neuronas de entrada y 1 de salida.
    self.fc2s = torch.nn.Linear(128, 1)

  def forward(self, state):
    # Pasar la entrada a través de la primera capa convolucional y aplicar la función de activación ReLU.
    x = self.conv1(state)
    x = F.relu(x)

    # Pasar el resultado a través de la segunda capa convolucional y aplicar ReLU.
    x = self.conv2(x)
    x = F.relu(x)

    # Pasar el resultado a través de la tercera capa convolucional y aplicar ReLU.
    x = self.conv3(x)
    x = F.relu(x)

    # Aplanar el resultado de las capas convolucionales en un vector.
    x = self.flatten(x)

    # Pasar el vector aplanado a través de la primera capa totalmente conectada y aplicar ReLU.
    x = self.fc1(x)
    x = F.relu(x)

    # Calcular los valores de acción (Q-values) a partir del vector resultante.
    action_values = self.fc2a(x)

    # Calcular el valor del estado (state value) a partir del mismo vector.
    state_value = self.fc2s(x)[0]

    # Devolver tanto los valores de acción como el valor del estado.
    return action_values, state_value

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Part 2 - Entrenamiento

### Estableciendo el entorno

In [4]:
class PreprocessAtari(ObservationWrapper):

  def __init__(self, env, height=42, width=42, crop=lambda img: img, dim_order='pytorch', color=False, n_frames=4):
    super(PreprocessAtari, self).__init__(env)

    # Configuración de los parámetros de preprocesamiento
    self.img_size = (height, width)  # Dimensiones a las que se redimensionará la imagen
    self.crop = crop  # Función para recortar la imagen
    self.dim_order = dim_order  # Orden de las dimensiones ('pytorch' o 'tensorflow')
    self.color = color  # Indica si se mantendrá la imagen en color o se convertirá a escala de grises
    self.frame_stack = n_frames  # Número de frames que se apilarán

    # Determinación del número de canales según si la imagen es a color o en escala de grises
    n_channels = 3 * n_frames if color else n_frames

    # Configuración de la forma del espacio de observación basado en el framework (PyTorch o TensorFlow)
    obs_shape = {'tensorflow': (height, width, n_channels), 'pytorch': (n_channels, height, width)}[dim_order]

    # Definición del espacio de observación con límites de valores entre 0 y 1
    self.observation_space = Box(0.0, 1.0, obs_shape)

    # Inicialización del buffer de frames como un arreglo de ceros
    self.frames = np.zeros(obs_shape, dtype=np.float32)

  def reset(self):
    # Reinicia el buffer de frames a ceros al comenzar un nuevo episodio
    self.frames = np.zeros_like(self.frames)

    # Reinicia el entorno y obtiene la primera observación
    obs, info = self.env.reset()

    # Actualiza el buffer con la primera observación
    self.update_buffer(obs)
    return self.frames, info

  def observation(self, img):
    # Recorta y redimensiona la imagen según la función 'crop' y las dimensiones configuradas
    img = self.crop(img)
    img = cv2.resize(img, self.img_size)

    # Convierte la imagen a escala de grises si no se mantiene en color
    if not self.color:
      if len(img.shape) == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Normaliza los valores de píxeles a un rango de 0 a 1
    img = img.astype('float32') / 255.

    # Actualiza el buffer de frames, desplazando los frames previos
    if self.color:
      self.frames = np.roll(self.frames, shift=-3, axis=0)
    else:
      self.frames = np.roll(self.frames, shift=-1, axis=0)

    # Inserta la imagen actualizada en el buffer de frames
    if self.color:
      self.frames[-3:] = img
    else:
      self.frames[-1] = img

    # Devuelve el buffer de frames actualizado
    return self.frames

  def update_buffer(self, obs):
    # Actualiza el buffer de frames usando la observación actual
    self.frames = self.observation(obs)

def make_env():
  # Crea el entorno de Atari KungFuMaster con una configuración determinística
  env = gym.make("KungFuMasterDeterministic-v0", render_mode='rgb_array')

  # Aplica el preprocesamiento personalizado al entorno
  env = PreprocessAtari(env, height=42, width=42, crop=lambda img: img, dim_order='pytorch', color=False, n_frames=4)

  # Devuelve el entorno con el preprocesamiento aplicado
  return env

env = make_env()

# Obtiene la forma del estado (observación) y el número de acciones posibles en el entorno
state_shape = env.observation_space.shape
number_actions = env.action_space.n

# Imprime la forma del estado y el número de acciones posibles
print("State shape:", state_shape)
print("Number actions:", number_actions)

# Imprime los nombres de las acciones en el entorno
print("Action names:", env.env.env.get_action_meanings())

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment KungFuMasterDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


State shape: (4, 42, 42)
Number actions: 14
Action names: ['NOOP', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'DOWNRIGHT', 'DOWNLEFT', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_action_meanings to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_action_meanings` for environment variables or `env.get_wrapper_attr('get_action_meanings')` that will search the reminding wrappers.
  logger.warn(


### Inicializar hiperparametros

In [5]:
learning_rate = 1e-4
discount_factor = 0.99
number_environments = 10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Implementacion de A3C class

In [6]:
class Agent():

  def __init__(self, action_size):
    # Configura el dispositivo a usar, prefiriendo GPU si está disponible, de lo contrario usa CPU.
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Tamaño del espacio de acciones (número de acciones posibles en el entorno).
    self.action_size = action_size

    # Inicializa la red neuronal que se utilizará para estimar los valores de acción y estado.
    self.network = Network(action_size).to(self.device)

    # Define el optimizador para ajustar los parámetros de la red durante el entrenamiento.
    self.optimizer = torch.optim.Adam(self.network.parameters(), lr=learning_rate)

  def act(self, state):
    # Si el estado tiene 3 dimensiones (un solo estado), lo convierte en una lista para ser compatible con la red.
    if state.ndim == 3:
      state = [state]

    # Convierte el estado a un tensor de PyTorch y lo envía al dispositivo (CPU o GPU).
    state = torch.tensor(state, dtype=torch.float32, device=self.device)

    # Pasa el estado por la red para obtener los valores de acción.
    action_values, _ = self.network(state)

    # Aplica la función softmax para convertir los valores de acción en una política (probabilidades).
    policy = F.softmax(action_values, dim=-1)

    # Selecciona una acción basada en las probabilidades de la política utilizando muestreo.
    return np.array([np.random.choice(len(p), p=p) for p in policy.detach().cpu().numpy()])

  def step(self, state, action, reward, next_state, done):
    # Convierte las variables de entrada a tensores de PyTorch y las envía al dispositivo.
    batch_size = state.shape[0]
    state = torch.tensor(state, dtype=torch.float32, device=self.device)
    next_state = torch.tensor(next_state, dtype=torch.float32, device=self.device)
    reward = torch.tensor(reward, dtype=torch.float32, device=self.device)
    done = torch.tensor(done, dtype=torch.bool, device=self.device).to(dtype=torch.float32)

    # Obtiene los valores de acción y estado actuales y futuros usando la red.
    action_values, state_value = self.network(state)
    _, next_state_value = self.network(next_state)

    # Calcula el valor objetivo del estado usando la fórmula de Bellman.
    target_state_value = reward + discount_factor * next_state_value * (1 - done)

    # Calcula la ventaja (advantage) comparando el valor objetivo con el valor estimado actual.
    advantage = target_state_value - state_value

    # Calcula la política y los logaritmos de la política usando softmax.
    probs = F.softmax(action_values, dim=-1)
    logprobs = F.log_softmax(action_values, dim=-1)

    # Calcula la entropía de la política, que mide la incertidumbre.
    entropy = -torch.sum(probs * logprobs, axis=-1)

    # Selecciona los logaritmos de las probabilidades de las acciones tomadas.
    batch_idx = np.arange(batch_size)
    logp_actions = logprobs[batch_idx, action]

    # Calcula la pérdida del actor utilizando la ventaja (advantage) y la entropía.
    actor_loss = -(logp_actions * advantage.detach()).mean() - 0.001 * entropy.mean()

    # Calcula la pérdida del crítico usando el error cuadrático medio entre el valor objetivo y el estimado.
    critic_loss = F.mse_loss(target_state_value.detach(), state_value)

    # La pérdida total es la suma de las pérdidas del actor y del crítico.
    total_loss = actor_loss + critic_loss

    # Resetea los gradientes del optimizador.
    self.optimizer.zero_grad()

    # Calcula los gradientes de la pérdida total.
    total_loss.backward()

    # Realiza un paso de optimización, ajustando los parámetros de la red.
    self.optimizer.step()

### Inicializar A3C agent

In [7]:
agent = Agent(number_actions)

### Evaluar el A3C agent en un numero de episodios

In [8]:
def evaluate(agent, env, n_episodes=1):
  episodes_rewards = []  # Lista para almacenar las recompensas totales de cada episodio

  # Bucle para evaluar el agente durante un número especificado de episodios
  for _ in range(n_episodes):
    state, _ = env.reset()  # Reinicia el entorno y obtiene el estado inicial
    total_reward = 0  # Inicializa la recompensa total del episodio en 0

    while True:  # Bucle que continuará hasta que el episodio termine
      action = agent.act(state)  # El agente selecciona una acción basada en el estado actual
      state, reward, done, info, _ = env.step(action[0])  # Ejecuta la acción en el entorno y obtiene el nuevo estado, recompensa, etc.
      total_reward += reward  # Suma la recompensa obtenida al total del episodio

      if done:  # Si el episodio ha terminado, sale del bucle
        break

    episodes_rewards.append(total_reward)  # Guarda la recompensa total del episodio

  return episodes_rewards  # Devuelve las recompensas totales de todos los episodios evaluados


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Manejar varios entornos de manera simultánea

In [9]:
class EnvBatch:

  def __init__(self, n_envs=10):
    # Inicializa un lote de entornos (n_envs) creando múltiples instancias del entorno de forma independiente.
    self.envs = [make_env() for _ in range(n_envs)]

  def reset(self):
    _states = []  # Lista para almacenar los estados iniciales de cada entorno.

    # Reinicia cada entorno y almacena su estado inicial.
    for env in self.envs:
      _states.append(env.reset()[0])

    # Convierte la lista de estados en un array de NumPy para un procesamiento más eficiente y la devuelve.
    return np.array(_states)

  def step(self, actions):
    # Ejecuta un paso en cada entorno utilizando las acciones correspondientes y recopila los resultados.
    next_states, rewards, dones, infos, _ = map(np.array, zip(*[env.step(a) for env, a in zip(self.envs, actions)]))

    # Revisa cada entorno para ver si ha terminado (done) y lo reinicia si es así.
    for i in range(len(self.envs)):
      if dones[i]:
        next_states[i] = self.envs[i].reset()[0]

    # Devuelve los siguientes estados, recompensas, indicadores de finalización (done) e información adicional.
    return next_states, rewards, dones, infos


### Entrenando el A3C agent

In [10]:
import tqdm  # Importa tqdm para mostrar una barra de progreso durante el bucle de entrenamiento.

# Inicializa un lote de entornos con un número específico de entornos.
env_batch = EnvBatch(number_environments)

# Reinicia todos los entornos en el lote y obtiene sus estados iniciales.
batch_states = env_batch.reset()

# Configura una barra de progreso para iterar desde 0 hasta 500 pasos de entrenamiento.
with tqdm.trange(0, 500) as progress_bar:
  for i in progress_bar:
    # El agente selecciona acciones basadas en los estados actuales del lote.
    batch_actions = agent.act(batch_states)

    # Ejecuta las acciones en el lote de entornos y obtiene los nuevos estados, recompensas, y si los episodios han terminado.
    batch_next_states, batch_rewards, batch_dones, _ = env_batch.step(batch_actions)

    # Escala las recompensas del lote por un factor de 0.01.
    batch_rewards *= 0.01

    # El agente realiza un paso de aprendizaje utilizando los estados, acciones, recompensas, nuevos estados e indicadores de finalización.
    agent.step(batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones)

    # Actualiza los estados actuales para el siguiente paso del bucle.
    batch_states = batch_next_states

    # Cada 1000 iteraciones, evalúa el desempeño del agente en el entorno y muestra la recompensa promedio.
    if i % 1000 == 0:
      print("Average agent reward: ", np.mean(evaluate(agent, env, n_episodes=10)))


/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment KungFuMasterDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
  0%|          | 0/500 [00:00<?, ?it/s]<ipython-input-6-c3e9a8e2f82a>:66: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(target_state_value.detach(), state_value)
<ipython-input-6-c3e9a8e2f82a>:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  state = torch.tensor(state, dtype=torch.float32, device=self.device)
  2%|▏         | 8/500 [00:38<28:25,  3.47s/it]  

Average agent reward:  460.0


100%|██████████| 500/500 [00:45<00:00, 10.99it/s]


## Part 3 - Ver los resultados

In [11]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder


# Crear el video
def show_video_of_model(agent, env):
  state, _ = env.reset()
  done = False
  frames = []
  while not done:
    frame = env.render()
    frames.append(frame)
    action = agent.act(state)
    state, reward, done, _, _ = env.step(action[0])
  env.close()
  imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, env)

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
